In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from xgboost import XGBRegressor
from xgboost import plot_importance
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from  sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ShuffleSplit
from sklearn.feature_selection import RFECV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from dateutil import parser
import random

Мы хотим получить сколько товара нужно произвести на две недели вперед исходя из продаж прошлого.

Сколько товаров у нас купять за неделю.

Какие признаки нужны:
- День недели
- Месяц
- Год
- Сезонность 03 по 08 месяц
- Цена товара
- Среднее количество продаж в конретный день недели
- Срежнее количество продаж за месяц
- Колличество продаж в прошлый день недели
- Колличество продаж в прошлый месяц
- Код позиции

Пробелмы которые нужно решить:
+ Найти признаки
- Обработать данные
- Найти оптимальны показатель точности модели
- Исправить размерность данных при выходе

In [2]:
df = pd.read_excel('/kaggle/input/time-series/DATA.xlsx')
df

,Номенклатура.Код,11.01.2021,Unnamed: 2,12.01.2021,Unnamed: 4,13.01.2021,Unnamed: 6,14.01.2021,Unnamed: 8,15.01.2021,...,08.05.2023,Unnamed: 1606,09.05.2023,Unnamed: 1608,10.05.2023,Unnamed: 1610,11.05.2023,Unnamed: 1612,12.05.2023,Unnamed: 1614
0,ПД0000022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1430.4,...,NaN,NaN,24942.40,140.0,62167.42,350.0,77312.34,448.0,NaN,NaN
1,ПД0000023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3259.56,12.0,3299.40,12.0
2,ПД0000024,NaN,NaN,10951.80,30.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,36378.00,60.0,NaN,NaN,7548.36,12.0
3,ПД0000235,NaN,NaN,NaN,NaN,57400.0,100.0,NaN,NaN,9216.0,...,NaN,NaN,5700.72,12.0,NaN,NaN,96769.44,192.0,NaN,NaN
4,ПД0000236,82255.44,444.0,652.88,4.0,51000.0,300.0,15669.12,96.0,4632.0,...,NaN,NaN,25804.08,108.0,NaN,NaN,97848.00,480.0,5900.16,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441,ПД0014507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
442,ПД0014508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
443,ПД0014593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
444,ПД0014682,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4853.55,30.0,NaN,NaN


Нахожу цену позиции

In [3]:
df = df.fillna(0)
columns = df.columns
len(columns)

list_sels = []
for column_price, column_value  in zip(columns[1:1615:2], columns[2:1615:2]):
    zipp = zip(df[column_price], df[column_value])
    for n, [cod_price, cod_value] in enumerate(zip(df[column_price], df[column_value])):
        # if int(cod_price) > 0:
            if cod_value == 0 or cod_price == 0:
                cod_price = 0
                cod_value = 0
                cod = df['Номенклатура.Код'][n]
                cod = cod.replace('ПД','')
                list_sels.append([cod, column_price, cod_price, cod_value]) 

            else:
            # print(cod_price, cod_value)
                cod_price = int(cod_price) / int(cod_value)
                cod = df['Номенклатура.Код'][n]
                cod = cod.replace('ПД','')
                list_sels.append([cod, column_price, cod_price, cod_value]) 
        # else:
        #     pass

df = pd.DataFrame(list_sels, columns=['cod', 'date','price', 'value'])
df

,cod,date,price,value
0,0000022,11.01.2021,0.000000,0.0
1,0000023,11.01.2021,0.000000,0.0
2,0000024,11.01.2021,0.000000,0.0
3,0000235,11.01.2021,0.000000,0.0
4,0000236,11.01.2021,185.259009,444.0
...,...,...,...,...
359917,0014507,12.05.2023,0.000000,0.0
359918,0014508,12.05.2023,0.000000,0.0
359919,0014593,12.05.2023,0.000000,0.0
359920,0014682,12.05.2023,0.000000,0.0


Убираем значения которые меньше нуля

In [4]:
df = df[(df['value'] > 0)]
df = df[(df['price'] > 0)]
df.reset_index(drop = True , inplace = True)

Далее убираю выбросы с помощью метода стандартного отклонения

In [5]:
df['value'].describe()

count    31972.000000
mean       880.829507
std       2466.631776
min          1.000000
25%         48.000000
50%        174.000000
75%        627.000000
max      52466.000000
Name: value, dtype: float64

In [6]:
# Threshold value to define the range for identifying outliers
threshold = 3

# Loop through all columns in the DataFrame, excluding the diagnosis
for column in df.loc[:, ~df.columns.isin(['price','date','cod'])]:  
    
    # Mean and STD of the column
    mean = df[column].mean()
    std = df[column].std()

    # Lower and Upper limits
    lower_limit = mean - threshold * std
    upper_limit = mean + threshold * std

    # Remove outliers
    df = df[(df[column] >= lower_limit) & (df[column] <= upper_limit)]

df['value'].describe()

count    31371.000000
mean       612.703994
std       1161.615790
min          1.000000
25%         48.000000
50%        162.000000
75%        588.000000
max       8272.000000
Name: value, dtype: float64

Разбиваем дату на отдельные признаки

In [7]:
seasonality_mounth = [3, 4, 5 ,6 , 7, 8]
list_seasonality = []

# Преобразование 'date' в правильный 'datetime' формат
# Извлечение дней, месяцев и годов из 'date'
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
# df['date'] = pd.to_datetime(df['date'], )
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

# Извлечение день недели для каждой даты
df['dayofweek'] = df['date'].dt.dayofweek


# Преобразование первоначальных ID товара, магазина и года(чтобы получить потом порядковый номер дня) в нормализованный вектор [0-max]
product_le = preprocessing.LabelEncoder()
product_le.fit(df['cod'].unique())
df['product_id_encoded'] = product_le.transform(df['cod'])

year_le = preprocessing.LabelEncoder()
year_le.fit(df['year'].unique())
df['year_encoded'] = year_le.transform(df['year'])

# Генерация порядкого номера дня для каждой даты
df['day_block_num'] = df.apply(lambda x: int(x['date'].strftime('%j')), axis=1)
df['day_block_num'] = df['day_block_num'] + 365 * df['year_encoded']

# Генерация порядкого номера месяца для каждой даты
df['month_block_num'] = df['month'] + 12 * df['year_encoded'] - 1

# Генерация нормализованного порядкого номера дня для каждой даты
day_block_num_le = preprocessing.LabelEncoder()
day_block_num_le.fit(df['day_block_num'].unique())
df['day_block_num_encoded'] = day_block_num_le.transform(df['day_block_num'])

#Генерация сизонности
for x in df['month']:
    if x in seasonality_mounth:
        x = 1
    else:
        x = 0
    list_seasonality.append(x)
df['seasonality'] = list_seasonality


# Сортировка данных по нормализованному порядковому номеру дня
df2 = df.sort_values(by='day_block_num_encoded', ascending=True)
df2['cat_id'] = 1
df2

,cod,date,price,value,day,month,year,dayofweek,product_id_encoded,year_encoded,day_block_num,month_block_num,day_block_num_encoded,seasonality,cat_id
0,0000236,2021-01-11,185.259009,444.0,11,1,2021,0,4,0,11,0,0,0,1
1,0000243,2021-01-11,105.183333,60.0,11,1,2021,0,8,0,11,0,0,0,1
2,0000244,2021-01-11,166.409091,22.0,11,1,2021,0,9,0,11,0,0,0,1
3,0000263,2021-01-11,371.307692,104.0,11,1,2021,0,23,0,11,0,0,0,1
4,0000266,2021-01-11,223.729339,484.0,11,1,2021,0,25,0,11,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31948,0000253,2023-05-12,441.270833,48.0,12,5,2023,4,16,2,862,28,806,1,1
31949,0000255,2023-05-12,155.372549,102.0,12,5,2023,4,17,2,862,28,806,1,1
31950,0000256,2023-05-12,247.777778,180.0,12,5,2023,4,18,2,862,28,806,1,1
31943,0000247,2023-05-12,138.285714,126.0,12,5,2023,4,11,2,862,28,806,1,1


Создаем признаки среднее колчиество продаж и цены за последний месяц

In [8]:
# Группировка данных по столбцам 'product_id_encoded', 'year', 'month', 'cat_id', суммирую 'value'
grouped_count = df2.groupby(['year', 'month','cat_id', 'product_id_encoded'])['value'].sum()

# Группировка данных по столбцам 'product_id_encoded', 'year', 'month', 'cat_id', извлекаю среднее значение 'price'
grouped_price = df2.groupby(['year', 'month', 'cat_id','product_id_encoded'])['price'].mean()

# Преобразование полученных группировок в формат DataFrame
multiindex_count = grouped_count.index.to_frame()
multiindex_price = grouped_price.index.to_frame()
temp_df = pd.DataFrame(multiindex_count[['year', 'month','cat_id', 'product_id_encoded']].values, columns=['year','month','cat_id', 'product_id_encoded'])
temp_df['count_per_month'] = grouped_count.values
temp_df['price_per_month'] = grouped_price.values

df3 = temp_df.copy(True)
df3

,year,month,cat_id,product_id_encoded,count_per_month,price_per_month
0,2021,1,1,0,444.0,115.805360
1,2021,1,1,1,394.0,177.178466
2,2021,1,1,2,156.0,378.071296
3,2021,1,1,3,1096.0,406.034854
4,2021,1,1,4,8094.0,174.972166
...,...,...,...,...,...,...
4560,2023,5,1,427,7546.0,138.769944
4561,2023,5,1,428,3000.0,141.700000
4562,2023,5,1,429,120.0,139.666667
4563,2023,5,1,430,60.0,161.766667


Создаем признаки среднее колчиество продаж и цены за последниюю неделю

In [9]:
# Группировка данных по столбцам 'day_block_num_encoded', 'year', 'month', 'seasonality', 'dayofweek', 'product_id_encoded' суммирую 'value'
grouped_count = df2.groupby(['day_block_num_encoded', 'year', 'month', 'seasonality', 'dayofweek', 'product_id_encoded'])['value'].sum()

# Группировка данных по столбцам 'day_block_num_encoded', 'year', 'month', 'seasonality', 'dayofweek', 'product_id_encoded' извлекаю среднее значение 'price'
grouped_price = df2.groupby(['day_block_num_encoded', 'year', 'month', 'seasonality', 'dayofweek', 'product_id_encoded'])['price'].mean()

# Преобразование полученных группировок в формат DataFrame
multiindex_count = grouped_count.index.to_frame()
multiindex_price = grouped_price.index.to_frame()
temp_df = pd.DataFrame(multiindex_count[['day_block_num_encoded', 'year', 'month', 'seasonality', 'dayofweek', 'product_id_encoded']].values, columns=['day_block_num_encoded', 'year', 'month', 'seasonality', 'dayofweek', 'product_id_encoded'])
temp_df['count_per_day'] = grouped_count.values
temp_df['price_per_day'] = grouped_price.values

df = temp_df.copy(True)
df

,day_block_num_encoded,year,month,seasonality,dayofweek,product_id_encoded,count_per_day,price_per_day
0,0,2021,1,0,0,4,444.0,185.259009
1,0,2021,1,0,0,8,60.0,105.183333
2,0,2021,1,0,0,9,22.0,166.409091
3,0,2021,1,0,0,23,104.0,371.307692
4,0,2021,1,0,0,25,484.0,223.729339
...,...,...,...,...,...,...,...,...
31366,806,2023,5,1,4,68,19.0,214.263158
31367,806,2023,5,1,4,69,24.0,333.125000
31368,806,2023,5,1,4,74,720.0,149.609722
31369,806,2023,5,1,4,75,88.0,242.000000


Далее объеденяем наши получившиеся признаки в один датафрейм

In [10]:
df = df.merge(df3, on=['month', 'year','product_id_encoded'])
df

,day_block_num_encoded,year,month,seasonality,dayofweek,product_id_encoded,count_per_day,price_per_day,cat_id,count_per_month,price_per_month
0,0,2021,1,0,0,4,444.0,185.259009,1,8094.0,174.972166
1,1,2021,1,0,1,4,4.0,163.000000,1,8094.0,174.972166
2,2,2021,1,0,2,4,300.0,170.000000,1,8094.0,174.972166
3,3,2021,1,0,3,4,96.0,163.218750,1,8094.0,174.972166
4,4,2021,1,0,4,4,20.0,231.600000,1,8094.0,174.972166
...,...,...,...,...,...,...,...,...,...,...,...
31366,805,2023,5,1,3,201,22.0,250.454545,1,22.0,250.454545
31367,805,2023,5,1,3,202,36.0,250.444444,1,36.0,250.444444
31368,805,2023,5,1,3,204,24.0,301.541667,1,24.0,301.541667
31369,805,2023,5,1,3,349,138.0,435.913043,1,138.0,435.913043


Содаем признак который отображает количество продаж по конкретному товару в предыдущий день недели

In [11]:
tdf = df.copy(True)
tdf.sort_values(by='day_block_num_encoded', ascending=False)
tdf.reset_index(drop=True, inplace=True)

tdf['lag_count_per_day'] = [0] * len(tdf)
size = len(tdf.index)
for ind in tdf.index:
    product_id = tdf.iloc[ind]['product_id_encoded']
    day_block_num_encoded = tdf.iloc[ind]['day_block_num_encoded']
    _temp_df = tdf[(tdf['product_id_encoded'] == product_id)
                        & (tdf['day_block_num_encoded'] < day_block_num_encoded)]
    _temp_df.sort_values(by='day_block_num_encoded', ascending=False)

    if len(_temp_df) != 0:
        tdf.at[ind, 'lag_count_per_day'] = _temp_df.iloc[0]['count_per_day']
df = tdf.copy(True)
df

,day_block_num_encoded,year,month,seasonality,dayofweek,product_id_encoded,count_per_day,price_per_day,cat_id,count_per_month,price_per_month,lag_count_per_day
0,0,2021,1,0,0,4,444.0,185.259009,1,8094.0,174.972166,0
1,1,2021,1,0,1,4,4.0,163.000000,1,8094.0,174.972166,444
2,2,2021,1,0,2,4,300.0,170.000000,1,8094.0,174.972166,444
3,3,2021,1,0,3,4,96.0,163.218750,1,8094.0,174.972166,444
4,4,2021,1,0,4,4,20.0,231.600000,1,8094.0,174.972166,444
...,...,...,...,...,...,...,...,...,...,...,...,...
31366,805,2023,5,1,3,201,22.0,250.454545,1,22.0,250.454545,220
31367,805,2023,5,1,3,202,36.0,250.444444,1,36.0,250.444444,108
31368,805,2023,5,1,3,204,24.0,301.541667,1,24.0,301.541667,108
31369,805,2023,5,1,3,349,138.0,435.913043,1,138.0,435.913043,100


Далее приводим данные к одному типу

In [12]:
df['day_block_num_encoded'] = df['day_block_num_encoded'].astype(np.int64)
df['dayofweek'] = df['dayofweek'].astype(np.int64)
df['product_id_encoded'] = df['product_id_encoded'].astype(np.int64)
df['lag_count_per_day'] = df['lag_count_per_day'].astype(np.int64)
df['year'] = df['year'].astype(np.int64)
df['month'] = df['month'].astype(np.int64)
df['seasonality'] = df['seasonality'].astype(np.int64)
df['count_per_day'] = df['count_per_day'].astype(np.int64)
df['price_per_day'] = df['price_per_day'].astype(np.int64)
df['count_per_month'] = df['count_per_month'].astype(np.int64)
df['price_per_month'] = df['price_per_month'].astype(np.int64)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31371 entries, 0 to 31370
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   day_block_num_encoded  31371 non-null  int64
 1   year                   31371 non-null  int64
 2   month                  31371 non-null  int64
 3   seasonality            31371 non-null  int64
 4   dayofweek              31371 non-null  int64
 5   product_id_encoded     31371 non-null  int64
 6   count_per_day          31371 non-null  int64
 7   price_per_day          31371 non-null  int64
 8   cat_id                 31371 non-null  int64
 9   count_per_month        31371 non-null  int64
 10  price_per_month        31371 non-null  int64
 11  lag_count_per_day      31371 non-null  int64
dtypes: int64(12)
memory usage: 2.9 MB


Создаем данные для предсказания на неделю вперед

In [13]:
DAYS_TO_PREDICT = 7
shelf_dict = dict()

categories = df['cat_id'].unique().tolist()
cat_product_dict = dict()
for category in categories:
    product_price = dict()
    products = df['product_id_encoded'].unique().tolist()
    products.sort()

    product_grouped = df.groupby(['product_id_encoded'])['price_per_day'].max()
    multiindex_price = product_grouped.index.to_frame()

    temp_df = pd.DataFrame(multiindex_price[['product_id_encoded']].values, columns=['product_id_encoded'])
    temp_df['price_per_day'] = product_grouped.values
    temp_df['product_id_encoded'] = [int(x) for x in temp_df['product_id_encoded']]
    temp_df.sort_values(by='price_per_day', ascending=True, inplace=True)

    for ind in temp_df.index:
        product_price[temp_df.iloc[ind]['product_id_encoded']] = temp_df.iloc[ind]['price_per_day']

    cat_product_dict[category] = product_price
shelf_dict['new'] = cat_product_dict

#     # **Генерация данных для прогноза для каждого уникального набора  витрина-категория-товар-цена
#     # **

#     # столбцы которые будут вносится в модель 
test_columns = ['day_block_num_encoded', 'dayofweek', 'cat_id', 'product_id_encoded', 'price_per_day', 'lag_count_per_day']
test_df = pd.DataFrame(columns=test_columns)
dayofweek = 0
last_day = df['day_block_num_encoded'].max()

#     # Генерация пар магазин-категория-товар-стоимость_товара со случайной продажой за последний день/2дня
for day in range(last_day + 1, last_day + 1 + DAYS_TO_PREDICT):
    for shelf in shelf_dict.keys():
        categories_as_keys = shelf_dict[shelf].keys()
        for category in categories_as_keys:
            products = shelf_dict[shelf][category].keys()
            prices = shelf_dict[shelf][category].values()
            array_size = len(products)
            lag_count_1 = [random.randint(0,1)] * array_size

            temp_df = pd.DataFrame(list(zip([day] * array_size, [dayofweek] * array_size, 
                                            [category] * array_size, products, prices, lag_count_1, )),columns=test_columns)
            test_df = pd.concat([test_df, temp_df])
    dayofweek += 1
    if dayofweek == 7:
        dayofweek = 0
test_df

# **Преобразование данных в соответствующие тип данных для обучение**

# Так как в ходе обработок, данные или массивы типов меняются на объекты, их стоит преобразовать в правильные форматы

# Тренировочные данные
df['day_block_num_encoded'] = df['day_block_num_encoded'].astype(np.int64)
df['dayofweek'] = df['dayofweek'].astype(np.int64)
df['product_id_encoded'] = df['product_id_encoded'].astype(np.int64)
df['lag_count_per_day'] = df['lag_count_per_day'].astype(np.int64)
df['price_per_day'] = df['price_per_day'].astype(np.int64)


# Данные для прогноза
test_df['day_block_num_encoded'] = test_df['day_block_num_encoded'].astype(np.int64)
test_df['dayofweek'] = test_df['dayofweek'].astype(np.int64)
test_df['product_id_encoded'] = test_df['product_id_encoded'].astype(np.int64)
test_df['lag_count_per_day'] = test_df['lag_count_per_day'].astype(np.int64)
test_df['price_per_day'] = test_df['price_per_day'].astype(np.int64)

df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31371 entries, 0 to 31370
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   day_block_num_encoded  31371 non-null  int64
 1   year                   31371 non-null  int64
 2   month                  31371 non-null  int64
 3   seasonality            31371 non-null  int64
 4   dayofweek              31371 non-null  int64
 5   product_id_encoded     31371 non-null  int64
 6   count_per_day          31371 non-null  int64
 7   price_per_day          31371 non-null  int64
 8   cat_id                 31371 non-null  int64
 9   count_per_month        31371 non-null  int64
 10  price_per_month        31371 non-null  int64
 11  lag_count_per_day      31371 non-null  int64
dtypes: int64(12)
memory usage: 2.9 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3024 entries, 0 to 431
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dty

Разбиваем датасет на тренировочные и тестовые признаки 

In [14]:
x = df[['day_block_num_encoded', 'dayofweek',
        'product_id_encoded',
        'lag_count_per_day', 'price_per_day'
        ]]
y = df[['count_per_day']]

x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    train_size=0.8, 
                                                    random_state=42)

Обучаем нашу модель

In [15]:
model = XGBRegressor(
        max_depth=10,
        n_estimators=500,
        random_state=42,
        learning_rate=0.3,
        )

model.fit(
    x_train, 
    y_train, 
    eval_set=[(x_train, y_train), (x_test, y_test)], 
    verbose=True, 
    early_stopping_rounds = 50)

# **Прогноз и проверка валидации**

# Прогноз для валидации модели
Y_valid_predicted = model.predict(x_test)
score = model.score(x_train, y_train)
print(score)

cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
scores = cross_val_score(model, x_train, y_train,cv=cv, scoring='r2')
print("Mean cross-validation score: %.2f" % scores.mean())

[0]	validation_0-rmse:1077.15586	validation_1-rmse:1184.45215
[1]	validation_0-rmse:935.96659	validation_1-rmse:1069.42304
[2]	validation_0-rmse:836.33852	validation_1-rmse:996.48747
[3]	validation_0-rmse:777.03641	validation_1-rmse:956.97697
[4]	validation_0-rmse:733.20246	validation_1-rmse:936.64122
[5]	validation_0-rmse:697.25197	validation_1-rmse:923.00861


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[6]	validation_0-rmse:670.76884	validation_1-rmse:917.16730
[7]	validation_0-rmse:644.77002	validation_1-rmse:911.00688
[8]	validation_0-rmse:629.37026	validation_1-rmse:905.67997
[9]	validation_0-rmse:611.14075	validation_1-rmse:904.99893
[10]	validation_0-rmse:598.40186	validation_1-rmse:901.94460
[11]	validation_0-rmse:587.38791	validation_1-rmse:900.42554
[12]	validation_0-rmse:569.98796	validation_1-rmse:898.06270
[13]	validation_0-rmse:560.12628	validation_1-rmse:897.50988
[14]	validation_0-rmse:547.63476	validation_1-rmse:897.10167
[15]	validation_0-rmse:539.99319	validation_1-rmse:897.56450
[16]	validation_0-rmse:524.05493	validation_1-rmse:896.22904
[17]	validation_0-rmse:515.86808	validation_1-rmse:897.49557
[18]	validation_0-rmse:509.70584	validation_1-rmse:897.12867
[19]	validation_0-rmse:505.67001	validation_1-rmse:897.24451
[20]	validation_0-rmse:502.43346	validation_1-rmse:896.75773
[21]	validation_0-rmse:498.73966	validation_1-rmse:896.48768
[22]	validation_0-rmse:491.9

Получаю прогноз на неделю вперед

In [16]:
x = test_df[['day_block_num_encoded', 'dayofweek',
        'product_id_encoded', 
        'lag_count_per_day', 'price_per_day'
        ]]
Y_predict = model.predict(x)
predicted_N_days_df = pd.DataFrame()
# predicted_N_days_df = x_train.copy(True)
predicted_N_days_df['predicted'] = Y_predict
predicted_N_days_df

,predicted
0,16.138514
1,-274.560944
2,16.138514
3,16.138514
4,18.138737
...,...
3019,3725.037842
3020,1006.893005
3021,1614.515503
3022,281.983185
